# AymaraSDK + OpenAI Minimal Example

This notebook demonstrates:
- Creating an eval with AymaraSDK
- Fetching eval prompts
- Calling OpenAI (real API) with those prompts
- Creating an eval run with the responses

## Requirements
- Set `OPENAI_API_KEY` and `AYMARA_SDK_API_KEY` in your environment or `.env` file.
- Install dependencies: `pip install openai aymara-sdk-python python-dotenv`

In [ ]:
# Environment and imports
import os

from dotenv import load_dotenv

load_dotenv()

import openai

from aymara_sdk import AymaraSDK
from aymara_sdk.types.eval_run_create_params import Response

## Set up API keys

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("OPENAI_API_KEY not set in environment.")
openai.api_key = OPENAI_API_KEY

## Instantiate the AymaraSDK client

In [ ]:
client = AymaraSDK()

## Create an eval

In [ ]:
eval_obj = client.evals.create(
    ai_description="Minimal SDK Example Eval",
    ai_instructions="Answer the prompts as best as you can.",
    eval_type="safety",
    name="minimal-example-eval",
    num_prompts=5
)
eval_id = eval_obj.eval_uuid
if not eval_id:
    raise RuntimeError("Eval creation failed.")
eval_obj

## Fetch prompts for the eval

In [ ]:
from aymara_sdk.lib.utils import wait_until

eval_obj = wait_until(
    client.evals.retrieve,
    predicate=lambda x: x.status == "finished",
    eval_uuid=eval_id
)
eval_obj

In [ ]:
prompts_response = client.evals.get_prompts(eval_id)
prompts = prompts_response.items
if not prompts:
    raise RuntimeError("No prompts found for eval.")


## Call OpenAI for each prompt and collect responses

In [ ]:
from typing import List

responses: List[Response] = []
for prompt in prompts:
    prompt_text = prompt.content
    prompt_uuid = prompt.prompt_uuid
    completion = openai.completions.create(
        model="gpt-4.1-nano-2025-04-14",
        prompt=[ prompt_text],
        max_tokens=256,
        temperature=0.7,
    )
    answer = completion.choices[0].text.strip()
    responses.append(Response(
        content=answer,
        prompt_uuid=prompt_uuid
    ))

## Create an eval run with the responses

In [ ]:
eval_run = client.eval_runs.create(
    eval_uuid=eval_id,
    responses=responses
)
eval_run_id = eval_run.eval_run_uuid
eval_run